In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [21]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'Cookie': 'AKA_A2=A; _ALGOLIA=anonymous-589227c0-ee82-4b6f-9443-bb6333065c8b; ajs_anonymous_id=10534979-fab5-4e02-9666-a364645e7c58; _gid=GA1.2.68085362.1718269442; WZRK_G=65ebb34640424f9c81446672c9497726; new_customer=true; nms_mgo_city=Ranchi; nms_mgo_state_code=JH; nms_mgo_pincode=834002; _ga_F1NJ1E2HJ2=GS1.1.1718271064.1.0.1718271098.0.0.0; _gat_UA-163452169-1=1; WZRK_S_88R-W4Z-495Z=%7B%22p%22%3A53%2C%22s%22%3A1718269441%2C%22t%22%3A1718271742%7D; _ga_XHR9Q2M3VV=GS1.1.1718269442.1.1.1718271743.37.0.0; _ga=GA1.1.89095287.1718269442; RT="z=1&dm=www.jiomart.com&si=a8e15a56-4c5a-4105-8bc1-034c90a463b8&ss=lxd19cx6&sl=1d&tt=15lw&obo=c&rl=1&nu=dvv9xwr&cl=1dgkg&ld=1dgl9&r=4yo67le9&ul=1dgla"'
}

### NOTE - 

we have used cookie here in header to give bigbasket our selected location info (cookie keep track your previous action and id tracking and login) 
hence after giving url request with our cookie, bigbasket will know that this cookie person is requesting and he was having this location from cookie tracking. hence will return data based on our previous action of location selection

for your cookie on a website - go to inspect, then network, now changing to any location on website will reflect your data fetch request of that location from website to load and can be seen in steps in Network. click the first fetch step - header in that request header portion will have your cookie who send the request for that location on that website. we will use that cookie in our code.

In [22]:
def extract_product_info(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    products = soup.find_all('div', class_= 'jm-col-4 jm-mt-base')

    #title = []
    name = []
    price = []
    discount_applied = []
    #quantity = []
    rating = []
    number_rating = []

    for product in products:
#         title_element = product.find('span', class_='Label-sc-15v1nk5-0 BrandName___StyledLabel2-sc-hssfrl-1 gJxZPQ keQNWn')
#         title_value = title_element.text.strip() if title_element else None
#         title.append(title_value)
        
        name_element = product.find('div', class_='plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80')
        name_value = name_element.text.strip() if name_element else None
        name.append(name_value)
        
        price_element = product.find('span', class_='jm-heading-xxs jm-mb-xxs')
        price_value = price_element.text.strip() if price_element else None
        price.append(price_value)
        
        discount_applied_element = product.find('span', class_='jm-badge')
        discount_applied_value = discount_applied_element.text.strip() if discount_applied_element else None
        discount_applied.append(discount_applied_value)
        
#         quantity_element = product.find('span', class_='Label-sc-15v1nk5-0 PackChanger___StyledLabel-sc-newjpv-1 gJxZPQ cWbtUx')
#         quantity_value = quantity_element.text.strip() if quantity_element else None
#         quantity.append(quantity_value)
        
#         rating_element = product.find('span', class_='Label-sc-15v1nk5-0 gJxZPQ')
#         rating_value = rating_element.text.strip() if rating_element else None
#         rating.append(rating_value)
        
#         number_rating_element = product.find('span', class_='Label-sc-15v1nk5-0 ReviewsAndRatings___StyledLabel-sc-2rprpc-1 gJxZPQ egHBA-d')
#         number_rating_value = number_rating_element.text.strip() if number_rating_element else None
#         number_rating.append(number_rating_value)

    return pd.DataFrame({
        #'Title': title,
        'Name': name,
        'Price': price,
        'Discount_applied': discount_applied,
#         'Quantity': quantity,
#         'Rating': rating,
#         'Number_rating': number_rating,
    })


In [24]:
category = {'fruits-vegetables': 219,
            'dairy-bakery': 61,
            'staples': 13,
            'snacks-branded-foods': 10,
            'beverages': 33,
            'personal-care': 91,
            'home-care': 36,
            'home-kitchen': 1687,
            'mom-baby-care': 2551,
            'beauty': 6607}

# Initialize an empty list to hold all DataFrames
all_data_frames = []

for cat_type in category:    
    all_pages_data = []  # Reset the list for each category
    
    url = f'https://www.jiomart.com/c/groceries/{cat_type}/{category[cat_type]}'
    response = requests.get(url, headers=header)
    if response.status_code == 200:
        page_data = extract_product_info(response.content)
        all_pages_data.append(page_data)
    else:
        print(f"Failed to retrieve page {page}")
            
    if all_pages_data:  # Check if there is any data before concatenating
        final_df = pd.concat(all_pages_data, ignore_index=True)
        final_df['Category'] = cat_type # Add a column with the category name
        all_data_frames.append(final_df)

In [25]:
JioMart_Data = pd.concat(all_data_frames , ignore_index=True)
JioMart_Data

,Name,Price,Discount_applied,Category
0,Onion 5 kg (Pack),₹190.00,None,fruits-vegetables
1,Okra 250 g,₹9.00,None,fruits-vegetables
2,Onion 1 kg (Pack),₹39.00,None,fruits-vegetables
3,Button Mushroom 200 g,₹55.00,None,fruits-vegetables
4,Tomato 1 kg,₹50.00,None,fruits-vegetables
...,...,...,...,...
115,ADULT ZONE Waterproof Rechargeable Personal Bo...,₹499.00,50% OFF,beauty
116,BUMTUM Baby Soap with goodness of calendula ex...,₹111.00,62% OFF,beauty
117,"ELDURO Premium Medium Adult Diaper Pant, 40 Co...","₹1,399.00",41% OFF,beauty
118,MEDIMIX Classic Soap to Maintain Healthy Skin ...,₹235.00,24% OFF,beauty


In [27]:
JioMart_Data.head(20)

,Name,Price,Discount_applied,Category
0,Onion 5 kg (Pack),₹190.00,None,fruits-vegetables
1,Okra 250 g,₹9.00,None,fruits-vegetables
2,Onion 1 kg (Pack),₹39.00,None,fruits-vegetables
3,Button Mushroom 200 g,₹55.00,None,fruits-vegetables
4,Tomato 1 kg,₹50.00,None,fruits-vegetables
5,Coccinia 250 g,₹9.00,None,fruits-vegetables
6,Muskmelon 700 g,₹55.00,None,fruits-vegetables
7,Watermelon Kiran 2 kg,₹49.00,None,fruits-vegetables
8,Carrot Orange 500 g,₹29.00,None,fruits-vegetables
9,Cabbage 1 pc (Approx 600 g - 1000 g),₹39.00,None,fruits-vegetables


In [26]:
#JioMart_Data.to_csv('JioMart Data Ranchi - JioMart.csv', index=False)

**Our logic for changing page number through loop in url will not work here because on loading url of any page, each time it begins from the first page - causing to repeatingly scrapping of only first page of each category**